In [1]:
import logging
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Флаг для управления видимостью логов retrieval
SHOW_RETRIEVAL_LOGS = False  # Установите True, чтобы видеть логи retrieval

# Настраиваем базовое логирование
if SHOW_RETRIEVAL_LOGS:
    logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s")
else:
    logging.basicConfig(level=logging.WARNING, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s")

# Настраиваем уровень логирования для различных модулей
if SHOW_RETRIEVAL_LOGS:
    # Включаем INFO логи от retriever модулей
    logging.getLogger("tplexity.retriever").setLevel(logging.INFO)
    logging.getLogger("tplexity.llm_client").setLevel(logging.INFO)
else:
    # Отключаем INFO логи от retriever модулей
    logging.getLogger("tplexity.retriever").setLevel(logging.WARNING)
    logging.getLogger("tplexity.llm_client").setLevel(logging.WARNING)

# Настраиваем логирование внешних библиотек
if SHOW_RETRIEVAL_LOGS:
    # Включаем DEBUG логи от httpx, чтобы видеть HTTP запросы/ответы и предупреждения от сервера LLM
    logging.getLogger("httpx").setLevel(logging.DEBUG)
    # Включаем логи от HuggingFace библиотек для просмотра предупреждений и сообщений от HF
    logging.getLogger("transformers").setLevel(logging.INFO)  # Основная библиотека HF
    logging.getLogger("huggingface_hub").setLevel(logging.INFO)  # Для работы с Hub
    logging.getLogger("huggingface").setLevel(logging.INFO)  # Общий логгер HF
else:
    logging.getLogger("httpx").setLevel(logging.WARNING)
    logging.getLogger("transformers").setLevel(logging.WARNING)
    logging.getLogger("huggingface_hub").setLevel(logging.WARNING)
    logging.getLogger("huggingface").setLevel(logging.WARNING)

# Всегда отключаем логи от других внешних библиотек (если не нужны)
logging.getLogger("sentence_transformers").setLevel(logging.WARNING)
logging.getLogger("pymorphy3").setLevel(logging.WARNING)

In [2]:
from tplexity.retriever.retriever_service import RetrieverService

service = RetrieverService(timeout=60)

/srv/nlp1/T-bank_NLP_/my_venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import json
from pathlib import Path

# Загружаем queries из файла
queries_path = project_root / "eval" / "data" / "queries.json"
with open(queries_path, "r", encoding="utf-8") as f:
    queries = json.load(f)

# Берем первый запрос
first_query = queries[1]["query"]
print(f"Запрос: {first_query}\n")

# Выполняем поиск (в Jupyter можно использовать await напрямую)
results = await service.search(query=first_query, use_rerank=False)

for result in results:
    print(result)


Запрос: Какие текущие тренды в области инвестиционных сообществ и социальных платформ для частных инвесторов в России?

('3a8a89a8-a968-4166-b890-3fcd73775fa7', 0.5, 'Сегмент «Социальные платформы и медиаконтент» вырос на 7% г/г во втором квартале, чему способствовало увеличение среднесуточного количества просмотров VK Клипов и VK Видео. Это ключевой сегмент для VK. Исторически его рентабельность была на уровне 30—40%. В прошлом году показатель был отрицательным. Во втором квартале 2025 года рентабельность сегмента составила 20%, что неплохо.\n\nОбразовательные технологии, экосистемные сервисы и прочие направления также показали уверенный рост выручки и положительные значения по рентабельности. \n\n📌** Что в итоге? **\n**\nКомментарий Аналитика Т-Инвестиций Марьяны Лазаричевой:**\n__Сегодняшние результаты VK соответствуют нашим ожиданиям по выручке и оказались немного сильнее по рентабельности. В целом, мы оцениваем результаты, как умеренно позитивные. Однако **на **__[**__акции VK__**

In [5]:
# Тест переформулировки запроса: берем 10 разных запросов, каждый отправляем в переформулировку 10 раз
# Это нужно для проверки, что переформулированный запрос всегда одинаковый (temperature=0)

import json
from pathlib import Path

# Загружаем queries из файла
queries_path = project_root / "eval" / "data" / "queries.json"
with open(queries_path, "r", encoding="utf-8") as f:
    all_queries = json.load(f)

# Берем 10 разных запросов
test_queries = [q["query"] for q in all_queries[:10]]

print(f"{'='*80}")
print(f"Тест детерминированности переформулировки запросов")
print(f"{'='*80}")
print(f"Будет протестировано {len(test_queries)} разных запросов")
print(f"Каждый запрос будет переформулирован 10 раз\n")

# Храним все переформулированные запросы для каждого исходного запроса
all_reformulations = {}

# Количество попыток переформулировки для каждого запроса
num_attempts = 100

# Обрабатываем каждый запрос
for query_idx, original_query in enumerate(test_queries, 1):
    print(f"\n{'─'*80}")
    print(f"Запрос {query_idx}/{len(test_queries)}: {original_query[:70]}...")
    print(f"{'─'*80}")
    
    reformulated_queries = []
    
    # Переформулируем запрос 10 раз
    for attempt in range(num_attempts):
        try:
            reformulated = await service._reformulate_query(original_query, messages=None)
            reformulated_queries.append(reformulated)
            if attempt == 0:
                print(f"  Попытка {attempt+1}: '{reformulated}'")
        except Exception as e:
            print(f"  ❌ Ошибка при попытке {attempt+1}: {type(e).__name__}: {str(e)}")
            reformulated_queries.append(None)
    
    all_reformulations[original_query] = reformulated_queries

# Анализируем результаты
print(f"\n{'='*80}")
print("Анализ результатов переформулировки:")
print(f"{'='*80}\n")

total_queries = len(test_queries)
queries_with_differences = []
queries_identical = []

for query_idx, (original_query, reformulated_list) in enumerate(all_reformulations.items(), 1):
    # Фильтруем None (ошибки)
    valid_reformulations = [r for r in reformulated_list if r is not None]
    
    if not valid_reformulations:
        print(f"❌ Запрос {query_idx}: все попытки завершились ошибкой")
        queries_with_differences.append((query_idx, original_query, reformulated_list))
        continue
    
    # Проверяем, все ли переформулировки одинаковые
    first_reformulated = valid_reformulations[0]
    all_same = all(r == first_reformulated for r in valid_reformulations)
    
    if all_same:
        print(f"✅ Запрос {query_idx}: все {len(valid_reformulations)} переформулировок идентичны")
        print(f"   Переформулированный: '{first_reformulated[:70]}...'")
        queries_identical.append(query_idx)
    else:
        print(f"❌ Запрос {query_idx}: обнаружены различия в переформулировках!")
        print(f"   Исходный запрос: '{original_query[:70]}...'")
        
        # Находим уникальные переформулировки
        unique_reformulations = list(set(valid_reformulations))
        print(f"   Найдено {len(unique_reformulations)} различных вариантов переформулировки:")
        
        for i, unique_reform in enumerate(unique_reformulations, 1):
            count = valid_reformulations.count(unique_reform)
            print(f"     Вариант {i} (встречается {count} раз): '{unique_reform}'")
        
        queries_with_differences.append((query_idx, original_query, reformulated_list))

# Итоговая статистика
print(f"\n{'='*80}")
print("Итоговая статистика:")
print(f"{'='*80}")
print(f"Всего протестировано запросов: {total_queries}")
print(f"✅ Запросов с идентичной переформулировкой: {len(queries_identical)}")
print(f"❌ Запросов с различиями в переформулировке: {len(queries_with_differences)}")

if queries_with_differences:
    print(f"\n⚠️  Запросы с различиями:")
    for query_idx, original_query, reformulated_list in queries_with_differences:
        print(f"   Запрос {query_idx}: '{original_query[:60]}...'")
else:
    print(f"\n✅ Все переформулировки детерминированы! Temperature=0 работает корректно.")

print(f"{'='*80}")


Тест детерминированности переформулировки запросов
Будет протестировано 10 разных запросов
Каждый запрос будет переформулирован 10 раз


────────────────────────────────────────────────────────────────────────────────
Запрос 1/10: Запущено ли сообщество для инвесторов в приложении Альфа-Инвестиций то...
────────────────────────────────────────────────────────────────────────────────
  Попытка 1: 'Доступно ли сообщество для инвесторов в приложении Альфа-Инвестиций на платформе iOS, или оно запущено только для Android?'

────────────────────────────────────────────────────────────────────────────────
Запрос 2/10: Какие текущие тренды в области инвестиционных сообществ и социальных п...
────────────────────────────────────────────────────────────────────────────────
  Попытка 1: 'Текущие тренды в инвестиционных сообществах и социальных платформах для частных инвесторов в России: анализ популярных направлений, инструментов и технологий, используемых в 2023–2024 годах.'

───────────────────

In [ ]:
import json
from tqdm import tqdm

# Параметры для расчета ONR@k
k = 5  # Количество возвращаемых результатов для оценки ONR@k
max_queries = 10  # Максимальное количество запросов для обработки

# Загружаем queries из файла
queries_path = project_root / "eval" / "data" / "queries.json"
with open(queries_path, "r", encoding="utf-8") as f:
    all_queries = json.load(f)

# Фильтруем queries: только те, где query_num равен 1, 2 или 3
filtered_queries = [q for q in all_queries if q.get("query_num") in [1]]
print(f"Всего запросов в файле: {len(all_queries)}")
print(f"Отфильтровано запросов: {len(filtered_queries)}")

# Сортируем для гарантии одинакового порядка при каждом запуске
# Сортируем по (id_channel, id_message, query_num) для детерминированности
filtered_queries.sort(key=lambda x: (x.get("id_channel", 0), x.get("id_message", 0), x.get("query_num", 0)))

# Ограничиваем количество запросов для обработки
queries_to_process = filtered_queries[:max_queries]
print(f"Будет обработано запросов: {len(queries_to_process)}\n")

# Счетчики
hits = 0
total_queries = len(queries_to_process)

print(f"Начинаем оценку ONR@{k} для {total_queries} запросов...\n")

# Обрабатываем каждый запрос
for i, query_data in enumerate(tqdm(queries_to_process, desc="Обработка запросов")):
    query_text = query_data["query"]
    expected_channel_id = query_data["id_channel"]
    expected_message_id = query_data["id_message"]
    
    try:
        # Выполняем поиск БЕЗ reranker
        results = await service.search(
            query=query_text,
            top_n=k,  # Возвращаем top_n результатов
            use_rerank=False
        )
        
        # Проверяем, есть ли совпадение среди результатов
        found_match = False
        for doc_id, score, text, metadata in results:
            if metadata:
                result_channel_id = metadata.get("channel_id")
                result_message_id = metadata.get("message_id")
                
                # Сравниваем id канала и id поста
                if (result_channel_id == expected_channel_id and 
                    result_message_id == expected_message_id):
                    found_match = True
                    break
        
        if found_match:
            hits += 1
            
    except Exception as e:
        import traceback
        error_msg = f"Ошибка при обработке запроса {i+1}: {type(e).__name__}: {str(e)}"
        print(error_msg)
        # Выводим traceback для более детальной информации
        traceback.print_exc()
        total_queries -= 1
        continue

# Вычисляем ONR@k
onr_at_k = hits / total_queries if total_queries > 0 else 0.0

print(f"\n{'='*60}")
print(f"Результаты оценки ONR@{k}:")
print(f"{'='*60}")
print(f"Всего обработано запросов: {total_queries}")
print(f"Найдено совпадений (hits): {hits}")
print(f"ONR@{k} = {onr_at_k:.4f} ({onr_at_k*100:.2f}%)")
print(f"{'='*60}")
